In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import pandas as pd
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import scipy
import torchvision.transforms as T
from PIL import Image
import cv2
import json
import glob
from tqdm.notebook import tqdm
     

In [2]:
IMAGE_SIZE = [224 , 224]

In [4]:
train_diretory = './Data/Training Data modified/Training Data modified/'
test_diretory = './Data/Testing Data modified/Testing Data modified/'
val_diretory = './Data/Validation Data modified/Vaildation Data modified/'

In [5]:
folders = glob('./Data/Training Data modified/Training Data modified/*')
len(folders)

5

In [ ]:
cwd = os.getcwd()

ROOT_DIR = os.path.join(cwd, "Data/Training Data modified/Training Data modified")

labels = {}

for folder in os.listdir(ROOT_DIR):
    for file in os.listdir(os.path.join(ROOT_DIR, folder)):
        if file.endswith(".jpg"):
            full_name = os.path.join(ROOT_DIR, folder, file)
            labels[full_name] = folder

files = labels.keys()

In [6]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")​device = torch.device('cuda' if torch.cuda.is_available() else "cpu")​dinov2_vits14.to(device)​transform_image = T.Compose([T.ToTensor(), T.Resize(244), T.CenterCrop(224), T.Normalize([0.5], [0.5])]

In [ ]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """
    img = Image.open(img)

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}
    
    with torch.no_grad():
      for i, file in enumerate(tqdm(files)):
        embeddings = dinov2_vits14(load_image(file).to(device))

        all_embeddings[file] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

    with open("all_embeddings.json", "w") as f:
        f.write(json.dumps(all_embeddings))

    return all_embeddings

In [ ]:
embeddings = compute_embeddings(files)

In [ ]:
from sklearn import svm

clf = svm.SVC(gamma='scale')
clf.summary()

In [ ]:

y = [labels[file] for file in files]

print(len(embeddings.values()))

embedding_list = list(embeddings.values())

clf.fit(np.array(embedding_list).reshape(-1, 384), y)

In [ ]:
input_file = "Data/Training Data modified/Training Data modified/"

new_image = load_image(input_file)

with torch.no_grad():
    embedding = dinov2_vits14(new_image.to(device))

    prediction = clf.predict(np.array(embedding[0].cpu()).reshape(1, -1))

    print("Predicted class: " + prediction[0])

Found 10000 images belonging to 5 classes.
2


FileNotFoundError: [WinError 3] The system cannot find the path specified: './Data/Testing Data modified/Test Data modified/'